<a href="https://colab.research.google.com/github/fcampos-personix/NLP/blob/main/GCP_STT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Install Packages and Set Credentials**

In [13]:
import os
credentials= "/content/drive/MyDrive/PersoniX/Dev/GCP/neon-partition-313318-ee8b12572f26.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= credentials

In [ ]:
pip install google-cloud-speech

## **Speech To Text Function**

In [9]:
def transcribe_gcs_speakers(gcs_uri):
    from google.cloud import speech_v1p1beta1 as speech

    client = speech.SpeechClient()
    audio = speech.RecognitionAudio(uri = gcs_uri)
    config = speech.RecognitionConfig( 
        language_code = 'en-US',
        enable_speaker_diarization=True, 
        diarization_speaker_count=2, #multiple speakers option
        enable_automatic_punctuation=True, #add punctuation and uppercase
        use_enhanced=True, #enhanced model activation
        model="video", #enhanced for audio extracted from videos
        )
    
    operation = client.long_running_recognize(config=config, audio=audio)

    print('Waiting for operation to complete...')

    response = operation.result(timeout=3000)
    result = response.results[-1]

    words_info = result.alternatives[0].words
    
    tag=1
    speaker=""

    for word_info in words_info:
        if word_info.speaker_tag==tag:
            speaker=speaker+" "+word_info.word

        else:
            print("speaker {}: {}".format(tag,speaker))
            tag=word_info.speaker_tag
            speaker=""+word_info.word

    print("speaker {}: {}".format(tag,speaker))

## **Upload File To GC Bucket**

In [14]:
def upload_to_bucket(name,filepath,bucket_name='recordings_discovery'):
    from google.cloud import storage
    # Setting credentials using the downloaded JSON file
    client = storage.Client.from_service_account_json(json_credentials_path=credentials)
    # Creating bucket object
    bucket = client.get_bucket(bucket_name)
    # Name of the object to be stored in the bucket
    object_name_in_gcs_bucket = bucket.blob(name)
    # Name of the object in local file system
    object_name_in_gcs_bucket.upload_from_filename(filepath)
    print('File uploaded successfully!')


In [15]:
upload_to_bucket("Kari_Paxton.wav", "/content/drive/MyDrive/PersoniX/Interviews/Interview Recordings/WAV/Keri_Paxton.wav" )

File uploaded successfully!


## **Transcription Task**

In [ ]:
#the file must be uploaded to the GC bucket first
transcript= transcribe_gcs_speakers("gs://personix_nn/recordings_discovery/Meeting Mike Agronin.wav")